In [1]:
import os
import copy
import json
import time
from typing import List

from CONST import SEED, POPULATION_SIZE, GENERATIONS, HISTORY_PATH
from neural_networks import FNN, CNN, CRNN
from utils import (set_seed, plot_confusion_matrix, make_predictions, 
                   calculate_metrics, save_metrics, save_history)
from algorithms import mu_lambda_es_evolve, de_best_1_bin_evolve, load_population, save_population

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv2D, MaxPooling2D, Reshape
from sklearn.model_selection import train_test_split

2024-01-29 20:24:58.834419: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 20:24:58.911450: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 20:24:58.912682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 20:25:00.776043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
set_seed(SEED)

In [3]:
if not os.path.exists(HISTORY_PATH):
    os.makedirs(HISTORY_PATH)

In [4]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

train_images, val_images, train_labels, val_labels = train_test_split(test_images, test_labels, test_size=2000, random_state=SEED)

In [5]:
class_names = [str(i) for i in range(10)]

models: List[Model] = [
    FNN('SGD'), FNN('ES'), FNN('DE'),
    CNN('SGD'), CNN('ES'), CNN('DE'),
    # CRNN('SGD'), CRNN('ES'), CRNN('DE')
]

for model in models:
    start = time.time()
    if model.info == 'SGD':
        model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
        history_obj = model.fit(train_images, train_labels, epochs=5, batch_size=64, shuffle=True)
        history = history_obj.history['accuracy']

    elif model.info == 'ES':
        model.build((1, 28, 28, 1))

        best_individual, pop, history = de_best_1_bin_evolve(population_size=POPULATION_SIZE, 
                                                             generations=GENERATIONS, 
                                                             model=model, 
                                                             F=1, 
                                                             CR=0.2, 
                                                             test_images=val_images,
                                                             test_labels=val_labels, 
                                                             variant='cross', 
                                                             start_pop=None)
        
        model.set_weights(best_individual)
        history = np.multiply(history, -1).tolist()

    elif model.info == 'DE':
        model.build((1, 28, 28, 1))

        best_individual2, pop, history = mu_lambda_es_evolve(mu=5, 
                                                             lambda_=5, 
                                                             sigma=0.1, 
                                                             generations=10, 
                                                             test_images=val_images,
                                                             test_labels=val_labels, 
                                                             model=model,  
                                                             variant='cross', 
                                                             start_pop=None)

        model.set_weights(best_individual)
        history = np.multiply(history, -1).tolist()

    stop = time.time()
    runtime = stop - start

    save_history(str(model), history, plot=True);

    predicted_labels = make_predictions(model, test_images)

    accuracy, precision, recall, conf_matrix = calculate_metrics(predicted_labels, np.argmax(test_labels, axis=1))
    save_metrics(str(model), accuracy=accuracy, precision=precision, recall=recall, time=runtime)
    # print(f'{str(model)} Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')
    # print('Confusion Matrix:')
    # print(conf_matrix)
    
    plot_confusion_matrix(conf_matrix, class_names, title=f'{str(model)}', save=True)

    model.save(f'models/{str(model)}', save_format='tf')  

2024-01-29 20:25:03.802747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 20:25:03.803277: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/5
125/125 [==============================] - 1s 4ms/step - loss: 1.8042 - accuracy: 0.5021
Epoch 2/5
125/125 [==============================] - 0s 4ms/step - loss: 1.1056 - accuracy: 0.7804
Epoch 3/5
125/125 [==============================] - 0s 3ms/step - loss: 0.7956 - accuracy: 0.8336
Epoch 4/5
125/125 [==============================] - 0s 3ms/step - loss: 0.6434 - accuracy: 0.8560
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
INFO:tensorflow:Assets written to: models/FNN_SGD/assets


INFO:tensorflow:Assets written to: models/FNN_SGD/assets


63/63 [==============================] - 0s 3ms/step
___________________
Cross-entropy Loss: 26459.421875
63/63 [==============================] - 0s 2ms/step
___________________
Cross-entropy Loss: 27607.00390625

--------------------------
Evolution - generation: 0
Individual number: 0_0
63/63 [==============================] - 0s 2ms/step
___________________
Cross-entropy Loss: 26459.421875
Fitness of working point: (26459.422,)
63/63 [==============================] - 0s 2ms/step
___________________
Cross-entropy Loss: 28989.21875
Individual number: 0_1
63/63 [==============================] - 0s 2ms/step
___________________
Cross-entropy Loss: 26459.421875
Fitness of working point: (26459.422,)
63/63 [==============================] - 0s 1ms/step
___________________
Cross-entropy Loss: 28819.06640625
63/63 [==============================] - 0s 2ms/step
___________________
Acc: -0.134

--------------------------
Evolution - generation: 1
Individual number: 1_0
63/63 [==============

TypeError: Object of type ndarray is not JSON serializable

<Figure size 640x480 with 0 Axes>

<Figure size 1100x1000 with 0 Axes>